### Remove DB

In [1]:
import os

if os.path.exists('example.db'):
    os.remove('example.db')

### Setup DB

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

from param_persist.sqlalchemy.models import Base


engine = create_engine('sqlite:///example.db')
Base.metadata.create_all(engine)

connection = engine.connect()
transaction = connection.begin()
session = Session(bind=connection)

### Param Class

In [3]:
import param
import numpy as np

class ExampleParamClass(param.Parameterized):
    number_field = param.Number(0.5, doc="A simple number field.")
    integer_field = param.Integer(1, doc="A simple integer field.")
    string_field = param.String("My String", doc="A simple string field.")
    bool_field = param.Boolean(False, doc="A simple boolean field.")
    

### Serializer

In [4]:
from param_persist.serialize.serializer import ParamSerializer

p = ExampleParamClass()
param_json = ParamSerializer.to_json(p)
print(param_json)

print('\n---------\n')

param_dict = ParamSerializer.to_dict(p)
print(param_dict)

{"class_path": "__main__.ExampleParamClass", "params": [{"name": "bool_field", "value": false, "type": "param.Boolean"}, {"name": "integer_field", "value": 1, "type": "param.Integer"}, {"name": "number_field", "value": 0.5, "type": "param.Number"}, {"name": "string_field", "value": "My String", "type": "param.parameterized.String"}]}

---------

{'class_path': '__main__.ExampleParamClass', 'params': [{'name': 'bool_field', 'value': False, 'type': 'param.Boolean'}, {'name': 'integer_field', 'value': 1, 'type': 'param.Integer'}, {'name': 'number_field', 'value': 0.5, 'type': 'param.Number'}, {'name': 'string_field', 'value': 'My String', 'type': 'param.parameterized.String'}]}


### Create Agent

In [5]:
from param_persist.agents.sqlalchemy_agent import SqlAlchemyAgent
from param_persist.sqlalchemy.models import InstanceModel, ParamModel

agent = SqlAlchemyAgent(engine=engine)

In [6]:
def show_database(db_session):
    print('Instances:')
    for parameterized_instance in db_session.query(InstanceModel).all():
        print(f'\t{parameterized_instance}')
    
    print('Parameters:')
    for parameter in db_session.query(ParamModel).all():
        print(f'\t{parameter}\n\t\t{parameter.value}')
    

### Save a Parameterized Object

In [7]:
show_database(session)

my_param = ExampleParamClass()
my_param_id = agent.save(my_param)

show_database(session)


Instances:
Parameters:
Instances:
	<Instance(id="b86fdff9-57c1-41f6-8969-8382dab7603d", class_path="__main__.ExampleParamClass")>
Parameters:
	<Param(id="2c7fb11d-a1f6-4ccb-9c16-95ca8c2c5336", instance_id="b86fdff9-57c1-41f6-8969-8382dab7603d")>
		{"name": "bool_field", "value": false, "type": "param.Boolean"}
	<Param(id="27172aff-50ed-4d2c-9b06-085603fe8a67", instance_id="b86fdff9-57c1-41f6-8969-8382dab7603d")>
		{"name": "integer_field", "value": 1, "type": "param.Integer"}
	<Param(id="0b1d4a6d-c8c2-43a7-9b44-4de671fc780b", instance_id="b86fdff9-57c1-41f6-8969-8382dab7603d")>
		{"name": "number_field", "value": 0.5, "type": "param.Number"}
	<Param(id="b8776f5e-9fea-447e-a914-b65aa8ff37df", instance_id="b86fdff9-57c1-41f6-8969-8382dab7603d")>
		{"name": "string_field", "value": "My String", "type": "param.parameterized.String"}



### Update a Parameterized Object

In [8]:
my_param.number_field = 2.2
my_param.bool_field = True

agent.update(my_param, my_param_id)

show_database(session)

Instances:
	<Instance(id="b86fdff9-57c1-41f6-8969-8382dab7603d", class_path="__main__.ExampleParamClass")>
Parameters:
	<Param(id="2c7fb11d-a1f6-4ccb-9c16-95ca8c2c5336", instance_id="b86fdff9-57c1-41f6-8969-8382dab7603d")>
		{"name": "bool_field", "value": true, "type": "param.Boolean"}
	<Param(id="27172aff-50ed-4d2c-9b06-085603fe8a67", instance_id="b86fdff9-57c1-41f6-8969-8382dab7603d")>
		{"name": "integer_field", "value": 1, "type": "param.Integer"}
	<Param(id="0b1d4a6d-c8c2-43a7-9b44-4de671fc780b", instance_id="b86fdff9-57c1-41f6-8969-8382dab7603d")>
		{"name": "number_field", "value": 2.2, "type": "param.Number"}
	<Param(id="b8776f5e-9fea-447e-a914-b65aa8ff37df", instance_id="b86fdff9-57c1-41f6-8969-8382dab7603d")>
		{"name": "string_field", "value": "My String", "type": "param.parameterized.String"}


### Load Parameterized Object

In [9]:
my_param_loaded = agent.load(my_param_id)

print(my_param_loaded.bool_field)
print(my_param_loaded.number_field)
print(my_param_loaded.integer_field)
print(my_param_loaded.string_field)

True
2.2
1
My String


### Delete Parameterized Object

In [10]:
show_database(session)
agent.delete(my_param_id)
show_database(session)

Instances:
	<Instance(id="b86fdff9-57c1-41f6-8969-8382dab7603d", class_path="__main__.ExampleParamClass")>
Parameters:
	<Param(id="2c7fb11d-a1f6-4ccb-9c16-95ca8c2c5336", instance_id="b86fdff9-57c1-41f6-8969-8382dab7603d")>
		{"name": "bool_field", "value": true, "type": "param.Boolean"}
	<Param(id="27172aff-50ed-4d2c-9b06-085603fe8a67", instance_id="b86fdff9-57c1-41f6-8969-8382dab7603d")>
		{"name": "integer_field", "value": 1, "type": "param.Integer"}
	<Param(id="0b1d4a6d-c8c2-43a7-9b44-4de671fc780b", instance_id="b86fdff9-57c1-41f6-8969-8382dab7603d")>
		{"name": "number_field", "value": 2.2, "type": "param.Number"}
	<Param(id="b8776f5e-9fea-447e-a914-b65aa8ff37df", instance_id="b86fdff9-57c1-41f6-8969-8382dab7603d")>
		{"name": "string_field", "value": "My String", "type": "param.parameterized.String"}
Instances:
Parameters:
